In [1]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
from math import tan, atan, radians, degrees, sin, cos, atan2

In [2]:
# by https://stackoverflow.com/questions/2891790/how-to-pretty-printing-a-numpy-array-without-scientific-notation-and-with-given
import contextlib

@contextlib.contextmanager
def printoptions(*args, **kwargs):
    original = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    try:
        yield
    finally: 
        np.set_printoptions(**original)
        

## constructing projection matrix

In [3]:
def construct_proj_matrix(H=1080, W=1914, fov=50.0, near_clip=1.5):
    # for z coord
    f = near_clip  # the near clip, but f in the book
    n = 10003.815  # the far clip, rounded value of median, after very weird values were discarded
    # x coord
    r = W * n * tan(radians(fov) / 2) / H
    l = -r
    # y coord
    t = n * tan(radians(fov) / 2)
    b = -t
    # x00 = 2*n/(r-l)
    x00 = H / (tan(radians(fov) / 2) * W)
    # x11 = 2*n/(t-b)
    x11 = 1 / tan(radians(fov) / 2)
    return np.array([
        [x00, 0, -(r + l) / (r - l), 0],
        [0, x11, -(t + b) / (t - b), 0],
        [0, 0, -f / (f - n), -f * n / (f - n)],
        [0, 0, -1, 0],
    ])

def calc_matrix_params(proj):
    x22 = proj[2, 2]
    x23 = proj[2, 3]
    n = -x23 / x22
    f = -x23 / (x22 - 1)
    return n, f

## loading data

In [4]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file

conn = visualization.get_connection()
cur = conn.cursor()
cur.execute("""SELECT snapshot_id, imagepath, proj_matrix, view_matrix, \
    ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)] as camera_pos, \
    ARRAY[st_x(camera_rot), st_y(camera_rot), st_z(camera_rot)] as camera_rot, \
    ARRAY[st_x(camera_direction), st_y(camera_direction), st_z(camera_direction)] as camera_direction \
  FROM snapshots \
  WHERE run_id = 6
  ORDER BY snapshot_id DESC \
  """)

results = []
for row in cur:
    res = dict(row)
    res['proj_matrix'] = np.array(res['proj_matrix'])
    res['view_matrix'] = np.array(res['view_matrix'])
    res['camera_pos'] = np.array(res['camera_pos'])
    res['camera_rot'] = np.array(res['camera_rot'])
    res['camera_direction'] = np.array(res['camera_direction'])
    results.append(res)

print('There are {} records'.format(len(results)))
projs = [i['proj_matrix'] for i in results]
views = [i['view_matrix'] for i in results]

# todo: estimate far plane by least squares. The rest is known. Before that, get rid of all outliers
# x00 = proj[0, 0]
# x11 = proj[1, 1]
x22s = np.array([proj[2, 2] for proj in projs])
x23s = np.array([proj[2, 3] for proj in projs])

ns = -x23s / x22s
fs = -x23s / (x22s - 1)

There are 33293 records


## inspecting projection matrix

In [ ]:
# showing plots with values
%matplotlib inline

fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$X_{22}$')
plt.hist(x22s, bins=1000)

fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$X_{23}$')
plt.hist(x23s, bins=1000)


fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$X_{22}$')
plt.ylabel('$X_{23}$')
plt.loglog(x22s, x23s, 'o', markersize=1)
plt.show()

In [ ]:
# showing weird matrices
with printoptions(precision=5, suppress=True):
    print('weird x22 value in proj: ')
    weird_projs = [proj for proj in projs if abs(proj[2, 2] - (-1)) < 1e-4]
    for weird_proj in weird_projs:
        print(weird_proj)
        print(calc_matrix_params(weird_proj))
    
    print('correct proj')
    print(projs[0])
    print(calc_matrix_params(projs[0]))

    print('weird x23 value in proj: ')
    weird_projs = [proj for proj in projs if proj[2, 3] > 1000]
    for weird_proj in weird_projs:
        print(weird_proj)
        print(calc_matrix_params(weird_proj))
    
    print('correct proj')
    print(projs[0])
    print(calc_matrix_params(projs[0]))



In [ ]:
# showing plots with n and f
%matplotlib inline
# axes[0].plot(x22s, np.zeros_like(x22s), 'x')
fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$N$')
plt.hist(ns, bins=1000)

fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$F$')
plt.hist(fs, bins=1000)

fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$N$')
plt.ylabel('$F$')
plt.loglog(ns, fs, 'o', markersize=1)
plt.show()

fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$N$')
plt.ylabel('$F$')
plt.plot(ns, fs, 'o', markersize=1)
plt.show()

In [ ]:
# filtered weird values
# this filters only very weird values
correct_projs = [proj for proj in projs if abs(proj[2, 2] - (-1)) > 1e-2 and proj[2, 3] < 1e3]
# this filters bassed on knowledge of near clip (far clip in terminilogy of equations, f)
correct_projs = [proj for proj in correct_projs if abs((-proj[2, 3] / (proj[2, 2] - 1)) - 1.5) < 5e-4]
x22s = np.array([proj[2, 2] for proj in correct_projs])
x23s = np.array([proj[2, 3] for proj in correct_projs])

ns = -x23s / x22s
fs = -x23s / (x22s - 1)

print('all values count: ', len(projs))
print('correct values count: ', len(correct_projs))
print('filtered count: ', len(projs) - len(correct_projs))

In [ ]:
# showing plots with values
%matplotlib inline

fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$X_{22}$')
plt.hist(x22s, bins=1000)

fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$X_{23}$')
plt.hist(x23s, bins=1000)


my_proj = construct_proj_matrix()

fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$X_{22}$')
plt.ylabel('$X_{23}$')
plt.loglog(x22s, x23s, 'o', markersize=1)
plt.plot(my_proj[2, 2], my_proj[2, 3], 'x', markersize=7)
plt.show()

fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$X_{22}$')
plt.ylabel('$X_{23}$')
plt.plot(x22s, x23s, 'o', markersize=1)
plt.plot(my_proj[2, 2], my_proj[2, 3], 'x', markersize=7)
plt.show()

dets = [np.linalg.det(proj) for proj in correct_projs]
fig = plt.figure(figsize=(14, 14))
plt.title('determinants')
plt.plot(dets, np.zeros_like(dets), 'o', markersize=1)
plt.plot(np.linalg.det(my_proj), 0, 'x', markersize=7)
plt.show()


In [ ]:
# showing plots with n and f
%matplotlib inline
# axes[0].plot(x22s, np.zeros_like(x22s), 'x')
fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$N$')
plt.hist(ns, bins=3000)

fig = plt.figure(figsize=(14, 14))
plt.yscale('log', nonposy='clip')
plt.title('$F$')
plt.hist(fs, bins=3000)

fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$N$')
plt.ylabel('$F$')
plt.loglog(ns, fs, 'o', markersize=1)
plt.show()

fig = plt.figure(figsize=(14, 14))
plt.title('both')
plt.xlabel('$N$')
plt.ylabel('$F$')
plt.plot(ns, fs, 'o', markersize=1)
plt.show()

In [ ]:
# fat clip, which I know precisely, is more noisy, near clip is more clear, taking value of the most frequent bin in histogram should suffice
bins = np.linspace(np.min(ns), np.max(ns), num=1000)
#print(bins)
inds = np.digitize(ns, bins)
print(inds)
print(np.sort(np.bincount(inds)))
print(np.argmax(np.bincount(inds)))
print(bins[np.argmax(np.bincount(inds))])
print(bins[np.argmax(np.bincount(inds)) - 1])
print(bins[np.argmax(np.bincount(inds)) + 1])

print('mean: ', np.mean(ns))
print('median: ', np.median(ns))


In [ ]:
# experimenting with generated matrices
my_proj = construct_proj_matrix()
proj = projs[0]
print('my_proj')
print(my_proj)
print('proj')
print(proj)
print('diff')
print(my_proj - proj)
print('diff sum')
print(np.sum(abs(my_proj - proj)))


### inspecting view matrices

In [16]:
def construct_view_matrix(camera_pos, camera_direction):
    view_matrix = np.zeros((4, 4))
    # view_matrix[0:3, 3] = camera_pos
    view_matrix[0:3, 0:3] = create_rot_matrix(camera_direction)
    view_matrix[3, 3] = 1
    
    trans_matrix = np.eye(4)
    trans_matrix[0:3, 3] = camera_pos
    
    return view_matrix @ trans_matrix

def create_rot_matrix(euler):
    x = np.radians(euler[0])
    y = np.radians(euler[1])
    z = np.radians(euler[2])
    Rx = np.array([[1, 0, 0],
                   [0, cos(x), -sin(x)],
                   [0, sin(x), cos(x)]], dtype=np.float)
    Ry = np.array([[cos(y), 0, sin(y)],
                   [0, 1, 0],
                   [-sin(y), 0, cos(y)]], dtype=np.float)
    Rz = np.array([[cos(z), -sin(z), 0],
                   [sin(z), cos(z), 0],
                   [0, 0, 1]], dtype=np.float)
    result = Rz @ Ry @ Rx
    return result

def calc_view_matrix_params(view):
    trans = view[0:3, 3]
    rot_mat = view[0:3, 0:3]
    return trans, rot_mat

In [17]:
view_matrix = results[0]['view_matrix']
pos = results[0]['camera_pos']
rot = results[0]['camera_rot']
direc = results[0]['camera_direction']
view_matrix_calc = construct_view_matrix(pos, direc)
print('view_matrix\n', view_matrix)
print('view_matrix_calc\n', view_matrix_calc)
print('diff\n', view_matrix - view_matrix_calc)
print('diff sum\n', np.sum(abs(view_matrix - view_matrix_calc)))

print('pos\n', pos)
print('pos length\n', np.linalg.norm(pos))
print('rot_vec\n', rot)


view_matrix
 [[  6.36678517e-01   7.71121144e-01   3.54826893e-03   1.61233813e+03]
 [ -5.27095795e-03  -2.49394914e-04   9.99986053e-01  -2.23262062e+01]
 [  7.71111310e-01  -6.36688352e-01   3.90576315e-03   1.23375745e+03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
view_matrix_calc
 [[  9.99938259e-01  -8.13831685e-05   1.11118052e-02  -1.97773285e+03]
 [ -6.81646447e-05   9.99909442e-01   1.34574204e-02  -4.57459779e+02]
 [ -1.11118942e-02  -1.34573470e-02   9.99847701e-01   3.99246136e+01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
diff
 [[ -3.63259742e-01   7.71202527e-01  -7.56353631e-03   3.59007098e+03]
 [ -5.20279330e-03  -1.00015884e+00   9.86528632e-01   4.35133573e+02]
 [  7.82223204e-01  -6.23231005e-01  -9.95941938e-01   1.19383283e+03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]]
diff sum
 5224.57269815
pos
 [-1978.02319336  -457.79467773    11.78612232]
pos length
 2030.34249158
rot_ve

In [ ]:
trans, rot_mat = calc_view_matrix_params(view_matrix)

In [ ]:
print('trans\n', trans)
print('trans length\n', np.linalg.norm(trans))
print('rot\n', rot)

In [ ]:
a, b, c = rot
rot_mat_calc = np.array([
    [cos(a)*cos(b), cos(a)*sin(b)*sin(c) - sin(a)*cos(c), cos(a)*sin(b)*cos(c) + sin(a)*sin(c)],
    [cos(a)*cos(b), cos(a)*sin(b)*sin(c) + cos(a)*cos(c), sin(a)*sin(b)*cos(c) - cos(a)*sin(c)],
    [-sin(b), cos(b)*sin(c), cos(b)*cos(c)],
])
print('rot_mat calculated\n', rot_mat_calc)

In [11]:
# Checks if a matrix is a valid rotation matrix.
import math

def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6


def rotationMatrixToEulerAngles(R): 
    assert(isRotationMatrix(R))
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
     
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([x, y, z])

In [13]:
euler = rotationMatrixToEulerAngles(view_matrix[0:3, 0:3])
print('euler angles from view matrix\n', euler)
print('euler angles degrees\n', np.degrees(euler))
print('rot_vec\n', rot)
print('direction_vec\n', direc)


euler angles from view matrix
 [-1.56466191 -0.88058475 -0.00827865]
euler angles degrees
 [-89.64852359 -50.45378972  -0.47433161]
rot_vec
 [ -0.22378571  -0.20330127  50.45510864]
direction_vec
 [-0.77112007  0.63667774 -0.00390579]
